In [2]:
import pandas as pd
import numpy as np
import torch
import os

In [40]:
from sklearn.model_selection import train_test_split

In [3]:
data='DATA'
raw_data_folder='RAW_DATA'
processed_data_folder='PROCESSED_DATA'

In [4]:
if data not in os.listdir(os.getcwd()):
    os.makedirs(os.path.join(data))
DATA_PATH=os.path.join(data)
if raw_data_folder not in os.listdir(os.path.join(DATA_PATH)):
    os.makedirs(os.path.join(DATA_PATH,raw_data_folder))
RAW_DATA_PATH=os.path.join(DATA_PATH,raw_data_folder)
if processed_data_folder not in os.listdir(os.path.join(DATA_PATH)):
    os.makedirs(os.path.join(DATA_PATH,processed_data_folder))
PROCESSED_DATA_PATH=os.path.join(DATA_PATH,processed_data_folder)

In [5]:
df=pd.read_csv(os.path.join(RAW_DATA_PATH,'Input Data Translated.csv'), encoding = 'latin-1')

In [6]:
data='DATA'
raw_data_folder='RAW_DATA'
processed_data_folder='PROCESSED_DATA'

if data not in os.listdir(os.getcwd()):
    os.makedirs(os.path.join(data))
DATA_PATH=os.path.join(data)
if raw_data_folder not in os.listdir(os.path.join(DATA_PATH)):
    os.makedirs(os.path.join(DATA_PATH,raw_data_folder))
RAW_DATA_PATH=os.path.join(DATA_PATH,raw_data_folder)
if processed_data_folder not in os.listdir(os.path.join(DATA_PATH)):
    os.makedirs(os.path.join(DATA_PATH,processed_data_folder))
PROCESSED_DATA_PATH=os.path.join(DATA_PATH,processed_data_folder)

In [7]:
df=pd.read_csv(os.path.join(RAW_DATA_PATH,'Input data split.csv'), encoding = 'latin-1')

In [8]:
df.head()

,Unnamed: 0,Short description,Description,Caller,Assignment group,Lang_Short_Description,Lang_Description,Final_description,Grp_change_1
0,0,login issue,-verified user details.(employee# & manager na...,spxjnwir pjlcoqds,GRP_0,en,en,login issue -verified user details.(employee# ...,7
1,1,outlook,\n\nreceived from: hmjdrvpb.komuaywn@gmail.com...,hmjdrvpb komuaywn,GRP_0,en,en,outlook received from: hmjdrvpb.komuaywn@gma...,7
2,2,cant log in to vpn,\n\nreceived from: eylqgodm.ybqkwiam@gmail.com...,eylqgodm ybqkwiam,GRP_0,en,en,cant log in to vpn received from: eylqgodm.y...,6
3,3,unable to access hr_tool page,unable to access hr_tool page,xbkucsvz gcpydteq,GRP_0,en,en,unable to access hr_tool page xbkucsvz gcpydteq,6
4,4,skype error,skype error,owlgqjme qhcozdfx,GRP_0,en,en,skype error owlgqjme qhcozdfx,6


In [9]:
df['group']=df['Grp_change_1'].copy()
val = df['group'].value_counts()
y = val[val < 30].index
df['group'] = df['group'].replace({x:'other' for x in y})

In [10]:
len(np.unique(df['group']))

44

In [11]:
print(f"Finally, there are {len(np.unique(df['group']))} labels for classification")

Finally, there are 44 labels for classification


In [12]:
text=df['Final_description'].values
labels=df['group'].values
unique_labels=np.unique(labels)

In [13]:
dict_index={} # {label: index}
dict_reverse_index={} #{index: label}
for index,value in enumerate(unique_labels):
    dict_index[value]=index
for k,v in dict_index.items():
    dict_reverse_index[v]=k
dict_index

{'0': 0,
 '1': 1,
 '2': 2,
 '3': 3,
 '4': 4,
 '5': 5,
 '6': 6,
 '7': 7,
 '8': 8,
 'GRP_1': 9,
 'GRP_10': 10,
 'GRP_11': 11,
 'GRP_12': 12,
 'GRP_13': 13,
 'GRP_14': 14,
 'GRP_15': 15,
 'GRP_16': 16,
 'GRP_17': 17,
 'GRP_18': 18,
 'GRP_19': 19,
 'GRP_2': 20,
 'GRP_20': 21,
 'GRP_22': 22,
 'GRP_24': 23,
 'GRP_25': 24,
 'GRP_26': 25,
 'GRP_28': 26,
 'GRP_29': 27,
 'GRP_3': 28,
 'GRP_30': 29,
 'GRP_31': 30,
 'GRP_33': 31,
 'GRP_34': 32,
 'GRP_4': 33,
 'GRP_40': 34,
 'GRP_41': 35,
 'GRP_42': 36,
 'GRP_45': 37,
 'GRP_5': 38,
 'GRP_6': 39,
 'GRP_7': 40,
 'GRP_8': 41,
 'GRP_9': 42,
 'other': 43}

In [14]:
labels_indexed=np.array([dict_index[k] for k in labels])
len(labels_indexed)

8500

In [15]:
from sklearn.utils.class_weight import compute_class_weight
balance=compute_class_weight(class_weight='balanced', classes=np.unique(labels_indexed), y=labels_indexed)

In [16]:
np.save(os.path.join(PROCESSED_DATA_PATH,'balanced_weights.npy'),balance)

### Tokenize features and OHK labels

#### 1. Tokenize features

In [17]:
from transformers import BertTokenizer
tokenizer=BertTokenizer.from_pretrained('bert-base-uncased')

In [18]:
text[:5]

array(['login issue -verified user details.(employee# & manager name) -checked the user name in ad and reset the password. -advised the user to login and check. -caller confirmed that he was able to login. -issue resolved. spxjnwir pjlcoqds',
       'outlook   received from: hmjdrvpb.komuaywn@gmail.com  hello team,  my meetings/skype meetings etc are not appearing in my outlook calendar, can somebody please advise how to correct this?  kind  hmjdrvpb komuaywn',
       'cant log in to vpn   received from: eylqgodm.ybqkwiam@gmail.com  hi  i cannot log on to vpn  best  eylqgodm ybqkwiam',
       'unable to access hr_tool page xbkucsvz gcpydteq',
       'skype error owlgqjme qhcozdfx'], dtype=object)

In [36]:
tokens=tokenizer(text.tolist(),max_length=512,truncation=True,padding='max_length',add_special_tokens=True,return_tensors='pt')

In [20]:
tokens.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [21]:
torch.save(tokens['input_ids'],os.path.join(PROCESSED_DATA_PATH,'input.pt'))
torch.save(tokens['attention_mask'],os.path.join(PROCESSED_DATA_PATH,'mask.pt'))

In [52]:
tokenizer.vocab_size

30522

In [22]:
torch.max(tokens['input_ids'])

tensor(29664)

In [22]:
final_labels=torch.tensor(labels_indexed,dtype=torch.int32)

In [23]:
torch.save(final_labels,os.path.join(PROCESSED_DATA_PATH,'labels.pt'))

### Train test split

In [43]:
TRAIN_PATH=os.path.join(PROCESSED_DATA_PATH,'TRAIN')
TEST_PATH=os.path.join(PROCESSED_DATA_PATH,'TEST')
if not os.path.exists(TRAIN_PATH):
    os.makedirs(TRAIN_PATH)
if not os.path.exists(TEST_PATH):
    os.makedirs(TEST_PATH)

In [44]:
X_train, X_test, y_train, y_test=train_test_split(text,labels_indexed,test_size=0.05,shuffle=True)

In [45]:
y_train.shape

(8075,)

In [46]:
X_train_tokens=tokenizer(X_train.tolist(),max_length=512,truncation=True,padding='max_length',add_special_tokens=True,return_tensors='pt')
#X_test_tokens=tokenizer(X_test.tolist(),max_length=512,truncation=True,padding='max_length',add_special_tokens=True,return_tensors='pt')

In [51]:
torch.max(X_train_tokens['input_ids'])

tensor(29664)

In [28]:
torch.save(X_train_tokens, os.path.join(TRAIN_PATH,'X_train_tokens.pt'))
torch.save(X_test_tokens, os.path.join(TEST_PATH,'X_test_tokens.pt'))

In [29]:
len(np.unique(y_train))

44

In [30]:
torch.save(torch.tensor(y_train,dtype=torch.int64), os.path.join(TRAIN_PATH,'y_train_tokens.pt'))
torch.save(torch.tensor(y_test,dtype=torch.int64), os.path.join(TEST_PATH,'y_test_tokens.pt'))